# How to Convert a Time Series to a Supervised Learning Problem in Python

https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

There are various different ways you can **reshape** a time series dataset to perform forecasting predictions via supervised machine learning:

- How to develop a function to transform a time series dataset into **a supervised learning dataset.**
- How to transform **univariate time series** data for machine learning.
- How to transform **multivariate time series** data for machine learning.

### Setting up a simple time series example

In [1]:
from pandas import DataFrame
df = DataFrame()
df['t'] = [x for x in range(10)]
df['t-1'] = df['t'].shift(1)
print(df)

   t  t-1
0  0  NaN
1  1  0.0
2  2  1.0
3  3  2.0
4  4  3.0
5  5  4.0
6  6  5.0
7  7  6.0
8  8  7.0
9  9  8.0


### The `series_to_supervised()` Function

We define a Python function named `series_to_supervised()` that takes a univariate or multivariate time series and frames (reshapes) it as a supervised learning dataset.

Input:

- **data:** Sequence of observations as a list or 2D NumPy array. Required.
- **n_in:** Number of lag observations as input (X). Values may be between [1..len(data)] Optional. Defaults to 1.
- **n_out:** Number of observations as output (y). Values may be between [0..len(data)-1]. Optional. Defaults to 1.
- dropnan: Boolean whether or not to drop rows with NaN values. Optional. Defaults to True.

Output:

- Pandas DataFrame of time series reshaped for supervised learning.

In [2]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.

	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.

	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()

	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names

	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)

	return agg

### One-Step Univariate Forecasting

Use a one lag time step (t-1) to predict the current time step (t):

- `n_in=1` - use the immediately preceding observation in the past as input

- `n_out=1` - predict the value at the current time step 

In [3]:
values = [x for x in range(10)]
data = series_to_supervised(values)
print(data)

   var1(t-1)  var1(t)
1        0.0        1
2        1.0        2
3        2.0        3
4        3.0        4
5        4.0        5
6        5.0        6
7        6.0        7
8        7.0        8
9        8.0        9


### Multi-Step or Sequence Forecasting

- Using past observations to forecast a sequence of future observations.

- This may be called **sequence forecasting** or **multi-step forecasting.**

Example:

- `n_in=2` - use two past observations as input

- `n_out=2` - predict the value at the current time step and the next future time step

The example below is still for a univariate time series.

In [4]:
values = [x for x in range(10)]
data = series_to_supervised(values, 2, 2)
print(data)

   var1(t-2)  var1(t-1)  var1(t)  var1(t+1)
2        0.0        1.0        2        3.0
3        1.0        2.0        3        4.0
4        2.0        3.0        4        5.0
5        3.0        4.0        5        6.0
6        4.0        5.0        6        7.0
7        5.0        6.0        7        8.0
8        6.0        7.0        8        9.0


### Multivariate Forecasting

We have observations of multiple different time series as input and we want to forecast one or more of them:

- The example below has two time series as input.

- We use the values of both time series e.g. at the previous time step (`n_in=1`) to predict the values of one or both series at the current time step (`n_out=1`). 

In [5]:
raw = DataFrame()
raw['ob1'] = [x for x in range(10)]
raw['ob2'] = [x for x in range(50, 60)]
values = raw.values
data = series_to_supervised(values)
print(data)

   var1(t-1)  var2(t-1)  var1(t)  var2(t)
1        0.0       50.0        1       51
2        1.0       51.0        2       52
3        2.0       52.0        3       53
4        3.0       53.0        4       54
5        4.0       54.0        5       55
6        5.0       55.0        6       56
7        6.0       56.0        7       57
8        7.0       57.0        8       58
9        8.0       58.0        9       59


### Example:

Reframing with 1 time step as input and 2 time steps as forecast sequence.

In [6]:
raw = DataFrame()
raw['ob1'] = [x for x in range(10)]
raw['ob2'] = [x for x in range(50, 60)]
values = raw.values
data = series_to_supervised(values, 1, 2)
print(data)

   var1(t-1)  var2(t-1)  var1(t)  var2(t)  var1(t+1)  var2(t+1)
1        0.0       50.0        1       51        2.0       52.0
2        1.0       51.0        2       52        3.0       53.0
3        2.0       52.0        3       53        4.0       54.0
4        3.0       53.0        4       54        5.0       55.0
5        4.0       54.0        5       55        6.0       56.0
6        5.0       55.0        6       56        7.0       57.0
7        6.0       56.0        7       57        8.0       58.0
8        7.0       57.0        8       58        9.0       59.0


### Summary

In this tutorial, you discovered how to reframe time series datasets as supervised learning problems with Python.

Specifically, you learned:

- About the Pandas shift() function and how it can be used to automatically define supervised learning datasets from time series data.

- How to reframe a univariate time series into one-step and multi-step supervised learning problems.

- How to reframe multivariate time series into one-step and multi-step supervised learning problems.

# For further studies:

How to Use XGBoost for Time Series Forecasting with Walk-Forward-Validation
https://machinelearningmastery.com/xgboost-for-time-series-forecasting/

How To Backtest Machine Learning Models for Time Series Forecasting
https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/